<a 
href="https://colab.research.google.com/github/adilo231/Python-for-machine-learning/blob/main/Exercices Global/Stroke prediction project.ipynb" 
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stroke Prediction

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'imblearn'

# Exploratory Data Analysis
## Analyse de la forme des données

- **variable target** : stroke
- **lignes et colonnes** : 5110, 12
- **types de variables** : qualitatives : 5, quantitatives : 7
- **Analyse des valeurs manquantes** : peut de valeurs NaN 3% de la colone bmi

In [ ]:
dataset = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [ ]:
df=dataset.copy()
df.shape


In [ ]:
df.dtypes.value_counts().plot.pie(autopct='%1.1f%%',startangle=90)
df.dtypes.value_counts()

In [ ]:
df.info()

**There are null values present in 'bmi'.**

In [ ]:
df.isna().sum()/dataset.shape[0]*100

seulement 3% des données «bmi» manquent. Nous avons remplacé les valeurs nulles de 'bmi' par la moyenne dans cette colonne.

In [ ]:
df.describe()

In [ ]:
df.corr()

# Data Visualization

### Heat Map Correlation

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Count Plot

In [ ]:
print(df.gender.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="gender")
plt.show()

In [ ]:
print(df.hypertension.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="hypertension")
plt.show()

In [ ]:
print(df.ever_married.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="ever_married")
plt.show()

In [ ]:
print(df.work_type.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="work_type")
plt.show()

In [4]:
print(df.Residence_type.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="Residence_type")
plt.show()

NameError: name 'df' is not defined

In [ ]:
print(df.smoking_status.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="smoking_status")
ax.set_xticklabels(ax.get_xticklabels(), fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
print(df.stroke.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="stroke")
plt.show()

    - Ci-dessus, vous pouvez voir que les femelles présentes dans notre ensemble de données sont plus élevées que les mâles.
    - D'en haut, cela montre que moins de personnes souffrent d'hypertension.
    - Le ratio vu d'en haut est d'environ 2: 1 pour être marié.
    - Beaucoup de gens travaillent dans le secteur privé.
    Le type de résidence est le même pour les personnes présentes dans notre jeu de données.
    - Beaucoup de gens n'ont jamais fumé de leur vie. Mais nous ne connaissons pas non plus le statut exact des inconnus dans notre ensemble de données.
    -D e la variable dépendante ci-dessus, nous avons vraiment moins de personnes qui ont subi un AVC. Mais cela signifie également que notre ensemble de données est déséquilibré. Nous devons probablement utiliser des techniques d'échantillonnage pour équilibrer les données.
    - Mais, commençons par tracer davantage pour voir comment nos données fonctionnent dans cet état.

In [ ]:
sns.pairplot(df,hue='stroke')

## **Distribution Plot** <a id="3.3"></a>

### **Avg. Glucose Level** <a id="3.3.1"></a>

In [ ]:
fig = plt.figure(figsize=(7,7))
sns.distplot(df.avg_glucose_level, color="green", label="avg_glucose_level", kde= True)
plt.legend()

1. Les taux de glucose normaux chez les adultes devraient se situer autour de 80-140. Par conséquent, la densité est plus élevée autour de cette plage. Ainsi, nous pouvons voir que nous avons beaucoup de gens qui ont un taux de glucose normal, donc ils ne souffrent pas de diabète.
2. La gamme 140-200 peut être considérée comme du pré-diabète. Mais, en regardant le graphique, nous pouvons voir que moins de personnes sont dans la zone de pré-diabète.
3. Tout ce qui dépasse 200 peut indiquer que la personne souffre de diabète. La densité est supérieure à celle du pré-diabète en regardant le graphique.

In [ ]:
fig = plt.figure(figsize=(7,7))
sns.distplot(df.bmi, color="orange", label="bmi", kde= True)
plt.legend()

1. Un IMC inférieur à 19 peut être considéré comme un poids insuffisant. En regardant notre graphique, peu de personnes souffrent d'insuffisance pondérale.

2. Un IMC entre 19 et 25 peut être considéré comme un poids normal. Nous avons une quantité relativement bonne de personnes qui ont un poids normal.

3. Un IMC supérieur à 25 peut être considéré comme la personne est probablement en surpoids ou obèse. Notre graphique montre que la densité est plus élevée autour de ces IMC.

In [ ]:
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["bmi"]) 
sns.distplot(df[df['stroke'] == 1]["bmi"]) 

plt.title('No Stroke vs Stroke by BMI', fontsize=15)
plt.xlim([10,100])
plt.show()

D'après le graphique, il montre que la densité de personnes en surpoids ayant subi un AVC est supérieure.

In [ ]:
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["avg_glucose_level"]) 
sns.distplot(df[df['stroke'] == 1]["avg_glucose_level"], )

plt.title('No Stroke vs Stroke by Avg. Glucose Level', fontsize=15)
plt.xlim([30,330])
plt.show()

D'après le graphique, il montre que la densité de personnes ayant un taux de glucose inférieur à 100 a subi davantage d'accidents vasculaires cérébraux.

In [ ]:
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["age"], color='green') # No Stroke - green
sns.distplot(df[df['stroke'] == 1]["age"], color='red') # Stroke - Red

plt.title('No Stroke vs Stroke by Age', fontsize=15)
plt.xlim([18,100])
plt.show()

D'après le graphique, on peut voir que la densité des personnes âgées de plus de 50 ans a davantage subi d'AVC.

## **Scatter Plot** <a id="3.4"></a>

In [ ]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data=df, x="age", y="avg_glucose_level", hue='gender')
graph.axhline(y= 150, linewidth=4, color='r', linestyle= '--')
plt.show()

D'après le graphique ci-dessus, nous pouvons voir que les personnes ayant un taux de glucose supérieur à 150 sont relativement moins nombreuses que celles ci-dessous. Ainsi, nous pouvons dire que les personnes de plus de 150 ans pourraient souffrir de diabète.

## **Violin Plot** <a id="3.5"></a>

In [ ]:
plt.figure(figsize=(13,13))
sns.set_theme(style="darkgrid")
plt.subplot(2,3,1)
sns.violinplot(x = 'gender', y = 'stroke', data = df)
plt.subplot(2,3,2)
sns.violinplot(x = 'hypertension', y = 'stroke', data = df)
plt.subplot(2,3,3)
sns.violinplot(x = 'heart_disease', y = 'stroke', data = df)
plt.subplot(2,3,4)
sns.violinplot(x = 'ever_married', y = 'stroke', data = df)
plt.subplot(2,3,5)
sns.violinplot(x = 'work_type', y = 'stroke', data = df)
plt.xticks(fontsize=9, rotation=45)
plt.subplot(2,3,6)
sns.violinplot(x = 'Residence_type', y = 'stroke', data = df)
plt.show()

# Preparation de donnees

In [ ]:
df =dataset.copy()
df =df.drop('id',1)
df.bmi.replace(to_replace=np.nan, value=dataset.bmi.mean(), inplace=True)
df = df.drop(df[df['gender']=='Other'].index,  0)
y = df.pop('stroke')
X = df
num_col = X.select_dtypes(exclude=['object']).columns
cat_col = X.select_dtypes(include=['object']).columns

In [ ]:
numerical_pipline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

In [ ]:
preprocessor = make_column_transformer((numerical_pipline, num_col),
                                      (categorical_pipline, cat_col))
X_transformed=preprocessor.fit_transform(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_transformed, y, test_size= 0.2, random_state= 0)
print("Number transactions x_train dataset: ", x_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions x_test dataset: ", x_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

In [ ]:

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

#  Selection des modeles <a id="9"></a>

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

In [ ]:
from tqdm import tqdm
models = []
models.append(['Logistic Regreesion', LogisticRegression(random_state=0)])
models.append(['SVM', SVC(random_state=0)])
models.append(['KNeighbors', KNeighborsClassifier()])
models.append(['GaussianNB', GaussianNB()])
models.append(['BernoulliNB', BernoulliNB()])
models.append(['Decision Tree', DecisionTreeClassifier(random_state=0)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
models.append(['XGBoost', XGBClassifier(eval_metric= 'error')])

lst_1= []

for m in tqdm(range(len(models))):
    lst_2= []
    model = models[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)  #Confusion Matrix
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_test, y_pred)  #ROC AUC Score
    precision = precision_score(y_test, y_pred)  #Precision Score
    recall = recall_score(y_test, y_pred)  #Recall Score
    f1 = f1_score(y_test, y_pred)  #F1 Score
 
    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_2.append(accuracies.std()*100)
    lst_2.append(roc)
    lst_2.append(precision)
    lst_2.append(recall)
    lst_2.append(f1)
    lst_1.append(lst_2)

In [ ]:
df = pd.DataFrame(lst_1, columns= ['Model', 'Accuracy', 'K-Fold Mean Accuracy', 'Std. Deviation', 'ROC AUC', 'Precision', 'Recall', 'F1'])
df.sort_values(by= ['Accuracy', 'K-Fold Mean Accuracy'], inplace= True, ascending= False)
df

# Réglage des modèles

In [ ]:
from sklearn.model_selection import GridSearchCV

*The **GridSearchCV** is a library function that is a member of sklearn's model_selection package. It helps to loop through predefined hyperparameters and fit your estimator (model) on your training set. So, in the end, you can select the best parameters from the listed hyperparameters.*

In [ ]:
grid_models = [
               (SVC(),[{'C':[0.25,0.5,0.75,1],'kernel':['linear', 'rbf'],'random_state':[0]}]), 
               (RandomForestClassifier(),[{'n_estimators':[50,100,150,200,250,300],'criterion':['gini','entropy'],'random_state':[0]}]), 
              (XGBClassifier(), [{'learning_rate': [0.01,0.025 ,0.05,0.075, 0.1,0.2,0.3], 'eval_metric': ['error']}])]

In [ ]:
models = []
models.append(['SVM', SVC(random_state=0,probability=True)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
models.append(['XGBoost', XGBClassifier(eval_metric= 'error')])

In [ ]:
from sklearn.model_selection import ShuffleSplit,StratifiedKFold
lst_1= []
modeles =[]

k=0
for i,j in tqdm(grid_models):
    lst_2= []
    
    grid = GridSearchCV(estimator=i,param_grid = j,cv = 10)
    grid.fit(x_train_res, y_train_res)
    model = grid.best_estimator_
    y_pred = model.predict(x_test)
   
    cm = confusion_matrix(y_test, y_pred)  #Confusion Matrix
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_test, y_pred)  #ROC AUC Score
    precision = precision_score(y_test, y_pred)  #Precision Score
    recall = recall_score(y_test, y_pred)  #Recall Score
    f1 = f1_score(y_test, y_pred)  #F1 Score
    
    modeles.append(model)
    lst_2.append(models[k][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_2.append(accuracies.std()*100)
    lst_2.append(roc)
    lst_2.append(precision)
    lst_2.append(recall)
    lst_2.append(f1)
    lst_1.append(lst_2)
    print(lst_2)
    k+=1


In [ ]:
df = pd.DataFrame(lst_1, columns= ['Model', 'Accuracy', 'K-Fold Mean Accuracy', 'Std. Deviation', 'ROC AUC', 'Precision', 'Recall', 'F1'])
df.sort_values(by= ['Accuracy', 'K-Fold Mean Accuracy'], inplace= True, ascending= False)
df

In [ ]:
modeles[3]

*En regardant la sortie après **GridSearch**, nous pouvons déterminer que la **RandomForest** semble la mieux adaptée au modèle.*

# Modèle après réglage des hyperparamètres

In [5]:
#Fitting RandomForest Model
classifier = modeles[2]
#classifier.fit(x_train_res, y_train_res)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)[:,1]
cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {roc_auc_score(y_test, y_prob)}')
print('Accuracy Score: ',accuracy_score(y_test, y_pred))

# Visualizing Confusion Matrix
plt.figure(figsize = (8, 5))
sns.heatmap(cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

# Roc AUC Curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)

sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

NameError: name 'modeles' is not defined

# Conclusion <a id="13"></a>

Donc, après les multiples visualisations de nos données et en passant par toutes les performances des modèles. J'ajuste les hyperparamètres à l'aide de GridSearch pour obtenir des modèles. Après cela, je suis arrivé à la conclusion que ***RandomForestClassifier*** est le meilleur modèle pour cet ensemble de données.